In [1]:
### starts from raw AMT ratings
import pandas as pd
import numpy as np
import csv
import os
%pprint off

Pretty printing has been turned OFF


In [2]:
path = '/Users/songheekim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/'
backups = next(os.walk(path))[1]
backups = [b for b in backups if b.startswith('backup')]
backups

['backup0_1_550', 'backup_210813', 'backup_210831', 'backup1_550-1050', 'backup3_0628', 'backup2_0610']

In [3]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [4]:
def return_res_catch_subj(backupfolder):
    
    path = '/Users/songheekim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/'
#     stimr = pd.read_csv(path+backupfolder+'/'+'wro1_stimuli.txt', delimiter="\t")
#     stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
#     stim['stim'] = stim['stim'].str[:-1]

    catchr = pd.read_csv(path+backupfolder+'/'+'wro1_catchresults.txt',delimiter="\t")
    catch = catchr.rename(columns={"response": "catch_response", "subject_id":"catch_subject_id"})

    resr = pd.read_csv(path+backupfolder+'/'+'wro1_results.txt', delimiter="\t")
    res = resr.drop(columns=['stimtype_id','sense'])

    subj_raw = pd.read_csv(path+backupfolder+'/'+'wro1_subjects.txt',delimiter="\t")
    subj = subj_raw.rename(columns={"id": "subject_id"})

    return (res, catch, subj)

In [5]:
#### basic check on each dataset ##### 
b=backups[2]

print('backup selected:', b)
print(os.listdir(path+'/'+b))

stimr = pd.read_csv(path+b+'/'+'wro1_stimuli.txt', delimiter="\t") # r means 'raw'
stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
stim['stim'] = stim['stim'].str[:-1]

ques = pd.read_csv(path+b+'/'+'wro1_questions.txt', delimiter="\t")
catchr = pd.read_csv(path+b+'/'+'wro1_catchresults.txt',delimiter="\t")
catch = catchr.rename(columns={"response": "catch_response", "subject_id":"catch_subject_id"})
print ("catch trial:", catch.shape[0])

resr = pd.read_csv(path+b+'/'+'wro1_results.txt', delimiter="\t")
res = resr.drop(columns=['stimtype_id','sense'])
#print ("main trial:", res.shape[0])
starttime = res.loc[res['id']==1, "start"].values[0]
endtime = res.loc[res['id']==res.shape[0], "end"].values[0]
print('This data contains ratings collected from', starttime, 'to', endtime)

subj_raw = pd.read_csv(path+b+'/'+'wro1_subjects.txt',delimiter="\t")
subj = subj_raw.rename(columns={"id": "subject_id"})
print('subject N:', subj.shape[0])

print('# of questions:', ques.shape[0])
print('# of catch HITs:', catch.shape[0])
print('# of HITs:     ', res.shape[0])
print('# of subjects: ', subj.shape[0])

check1 = catch.shape[0] == res.shape[0]
check2 = res.shape[0] >= subj.shape[0] 
check3 = res['subject_id'].equals(other=catch['catch_subject_id'])

if check1==0:
    print ('# of catch trials != # of trials')
elif check2==0:
    print ('# of subjects are more than # of HITs')
elif check3==0:
    print('subject_id are not the same in trials and catchtrials')

print ('=============================================') 


backup selected: backup_210831
['.DS_Store', 'wro1_stimuli.txt', 'wro1_subjects.txt', 'wro1_catchresults.txt', 'wro1_results.txt', 'wro1_questions.txt']
catch trial: 1518
This data contains ratings collected from 2021-08-24 12:37:49 to 2021-08-28 13:11:00
subject N: 874
# of questions: 72
# of catch HITs: 1518
# of HITs:      1518
# of subjects:  874


In [8]:
######catch trial info
catch1 = 'being an animal living on the surface of sun'
catch2 = 'being something about which you are supposed to answer questions in this assignment'
catch3 = 'being an object that is smaller than a shoe box and larger than a mountain'
catch4 = 'being something you have thought about within the last couple minutes'
catch5 = 'being used to check whether you are carefully reading the questions'
catch_answers = ['0/7','6','0','6','6']

# def count_correct_answers(list1):
#     catch_answers = ['0/7','6','0','6','6']
#     if len(list1) != len(catch_answers):
#         print ('responses have different lengths!')
#         return
#     correct_answer = 0
#     if list1[0] in catch_answers[0].split('/'):
#         correct_answer += 1
#     for i in range(1,len(list1)):
#         iscorrect = list1[i]==catch_answers[i]
#         correct_answer += iscorrect        
#     return correct_answer

def count_incorrect_answers(list1):
    catch_answers = ['0/7','6','0','6','6']
    if len(list1) != len(catch_answers):
        print ('responses have different lengths!')
        return
    correct_answer = 0
    if list1[0] in catch_answers[0].split('/'):
        correct_answer += 1
    for i in range(1,len(list1)):
        iscorrect = list1[i]==catch_answers[i]
        correct_answer += iscorrect        
    return (5-correct_answer)

count_incorrect_answers(['1', '6', '0', '6', '6'])

1

In [9]:
def merge_tables(backupfolder): 
    '''function that merges results table, catch table, and subject table 
       backupfolder = the rating folder of interest'''
    [res, catch, subj] = return_res_catch_subj(backupfolder)
    
    res_all = pd.merge(left=res, right=subj, on='subject_id')
    res_all = res_all.sort_values(by=['id'])
    print('result df:', res.shape)
    print('subject df:', subj.shape)
    print('df returned:', res_all.shape)
    
    catch = catch.drop(columns='id') 
    res_all = pd.merge(left=res_all, right=catch, left_on="id", right_on="results_id")
    
    #print(res_all.shape)
    #print(catch.shape)
    print ('=========================================================')
    
    return res_all

result df: (1518, 9)
subject df: (874, 6)
df returned: (1518, 14)


,id,stim_id,subject_id,response,start,end,rt,buttonpress,turkcode,subject_num,gender,age,education,occupation,results_id,catch_subject_id,catch_response
0,1,294,1,0_0_0_0_0_0_0_0_0_0_0_0_2_0_0_0_0_0_0_0_0_0_3_...,2021-08-24 12:37:49,2021-08-24 12:43:52,5841_1584_1808_2144_2008_2680_3416_11448_11423...,2_1_1_1_1_1_1_1_3_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,7467737,ADJ9I7ZBFYFH7,m,32,13.0,Security at a warehouse,1,1,0_6_0_6_6
1,2,208,2,0_0_0_1_3_0_0_0_0_0_1_2_0_0_0_1_3_0_0_0_0_0_0_...,2021-08-24 12:36:28,2021-08-24 12:49:54,5114_2594_2164_8210_26407_2706_9729_11037_3966...,1_1_1_2_1_1_4_1_1_1_2_2_1_2_1_1_5_1_1_1_1_1_1_...,5481024,A3NKZMF7M4X0ZP,m,27,16.0,NaN,2,2,0_6_0_6_6
2,3,40,3,3_0_0_0_1_0_1_6_6_6_3_2_1_6_0_0_0_0_0_0_0_3_0_...,2021-08-24 12:47:34,2021-08-24 12:52:58,4728_1777_1777_1864_11745_2536_4121_4793_4040_...,2_1_1_1_2_1_1_2_1_3_1_1_3_1_1_1_2_1_1_1_1_1_1_...,3663884,A2OG7XGWOX5ZEF,m,33,14.0,I work at a large retail store moving items an...,3,3,0_6_0_6_6
3,4,57,4,5_0_0_0_0_0_0_4_6_4_1_0_0_6_0_0_0_0_0_0_0_0_0_...,2021-08-24 12:49:04,2021-08-24 13:00:17,13362_2288_3391_3672_4813_3144_5584_6477_7000_...,2_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_1_1_...,1561528,A170R0WAZC31WC,f,51,15.0,Variable data tech for print on demand,4,4,0_6_0_6_6
4,5,22,5,3_3_0_2_0_0_1_3_6_1_1_0_2_4_1_2_1_0_0_0_0_3_2_...,2021-08-24 13:23:42,2021-08-24 13:31:44,7343_3488_3920_7568_2992_3808_4968_6368_3896_2...,2_1_1_1_1_1_1_2_1_1_1_1_1_1_1_1_2_1_2_1_1_2_1_...,1183680,AAXX5LDVJ32F8,m,49,12.0,unemployed,5,5,0_6_0_6_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,1514,254,872,5_3_1_2_4_4_3_1_1_2_2_1_0_2_6_3_2_4_3_1_1_1_3_...,2021-08-28 09:38:19,2021-08-28 09:48:43,3744_2720_2944_5064_5416_10400_4312_4416_3016_...,1_1_1_1_1_1_1_2_1_1_1_1_1_1_1_1_3_2_1_1_1_1_1_...,7910923,ALTSDQ7JAWE7E,f,39,16.0,AR Specialist for my full time job and office ...,1514,872,0_6_0_6_6
1514,1515,45,873,6_3_6_3_2_6_2_3_3_6_3_3_6_6_3_3_2_6_3_2_6_2_3_...,2021-08-28 09:44:10,2021-08-28 10:01:13,5328_3789_2640_3636_15144_3160_2187_3971_3087_...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,6194369,A1N35VV9QURKHX,f,45,14.0,Information Data Entry,1515,873,7_6_0_6_6
1515,1516,35,645,1_0_1_0_0_0_0_0_5_0_1_0_5_3_0_0_1_1_0_0_1_3_5_...,2021-08-28 10:18:07,2021-08-28 10:32:48,6088_3116_5168_33664_3972_4848_3268_3456_12468...,1_1_1_1_1_2_2_1_1_1_1_1_1_2_1_1_2_1_1_1_2_1_1_...,6822510,A3O0ZQMDKOKAMR,f,44,12.0,I am a fulltime Decorator for a retail grocery...,1516,645,0_6_0_6_6
1516,1517,29,577,5_0_0_0_0_0_0_1_6_2_3_0_2_5_6_6_1_5_0_0_1_1_1_...,2021-08-28 10:37:50,2021-08-28 10:53:09,4344_2234_2390_9221_20266_128198_21287_7657_96...,1_1_1_1_1_1_1_1_1_1_1_1_2_1_1_1_2_1_1_1_1_1_1_...,3616355,A49A0FF6ZXYDU,f,53,16.0,library staff,1517,577,0_6_0_6_6


In [19]:
# res_combined = pd.concat([res_all1, res_all2], ignore_index=True)
# res_combined['id'] = list(range(1, res_combined.shape[0]+1))

# print(res_all1.shape)
# print(res_all2.shape)
# print(res_combined.shape)
# res_combined

res_combined = merge_tables(b)
res_combined

##check whether there is a duplicate row
duplicates = res_combined.duplicated()
duplicates = duplicates.index[duplicates]
len(duplicates)


#### add lemma column
stimr = pd.read_csv(path+b+'/'+'wro1_stimuli.txt', delimiter="\t")
stim = stimr[stimr['id'].notna()][["id", "stim"]].reset_index(drop=True).astype({"id": int})
stim['stim'] = stim['stim'].str[:-1]
stim.rename(columns={'id':'stim_id'}, inplace=True)
stim

res_combined = pd.merge(left=res_combined, right=stim, on="stim_id").sort_values(by="id")
res_combined = rearrange_col(res_combined, 'stim', 'stim_id')
res_combined


##### mark bad subjects (updated)
res_combined['catch_response_split'] = res_combined['catch_response'].apply(lambda x:x.split('_'))
catch_series = res_combined['catch_response_split'].tolist()
#correct_answer = [count_correct_answers(l) for l in catch_series]
incorrect_answer = [count_incorrect_answers(l) for l in catch_series]
res_combined['incorrectN'] = incorrect_answer

res_combined.loc[res_combined['incorrectN']>=3, 'badsubject'] = 'Y'
res_combined.loc[res_combined['incorrectN']<3, 'badsubject'] = 'N'
res_bad = res_combined.loc[res_combined['badsubject']=='Y']
print('# of bad subject:', res_bad.shape[0], 'out of', res_combined.shape[0])

res_combined['response_split'] = res_combined['response'].apply(lambda x:x.split('_'))
res_combined['RT_split'] = res_combined['rt'].apply(lambda x:x.split('_'))
res_combined['buttonpress_split'] = res_combined['buttonpress'].apply(lambda x:x.split('_'))

res_combined = rearrange_col(res_combined, "response_split", "response")
res_combined = rearrange_col(res_combined, "RT_split", "rt")
res_combined = rearrange_col(res_combined, "buttonpress_split", "buttonpress")

for c in ['catch_response', 'response', 'rt', 'buttonpress']:
    res_combined.drop(columns=[c], inplace=True)
    
res_combined.rename(columns={'catch_response_split': 'catch_response', 'response_split':'response', 'RT_split':'RT', 'buttonpress_split':'buttonpress'}, inplace=True)

res_combined

result df: (1518, 9)
subject df: (874, 6)
df returned: (1518, 14)
# of bad subject: 11 out of 1518


,id,stim_id,stim,subject_id,response,start,end,RT,buttonpress,turkcode,subject_num,gender,age,education,occupation,results_id,catch_subject_id,catch_response,incorrectN,badsubject
0,1,294,imply,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...",2021-08-24 12:37:49,2021-08-24 12:43:52,"[5841, 1584, 1808, 2144, 2008, 2680, 3416, 114...","[2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, ...",7467737,ADJ9I7ZBFYFH7,m,32,13.0,Security at a warehouse,1,1,"[0, 6, 0, 6, 6]",0,N
5,2,208,thaw,2,"[0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, ...",2021-08-24 12:36:28,2021-08-24 12:49:54,"[5114, 2594, 2164, 8210, 26407, 2706, 9729, 11...","[1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 2, 2, 1, 2, 1, ...",5481024,A3NKZMF7M4X0ZP,m,27,16.0,NaN,2,2,"[0, 6, 0, 6, 6]",0,N
10,3,40,wriggle,3,"[3, 0, 0, 0, 1, 0, 1, 6, 6, 6, 3, 2, 1, 6, 0, ...",2021-08-24 12:47:34,2021-08-24 12:52:58,"[4728, 1777, 1777, 1864, 11745, 2536, 4121, 47...","[2, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 3, 1, 1, ...",3663884,A2OG7XGWOX5ZEF,m,33,14.0,I work at a large retail store moving items an...,3,3,"[0, 6, 0, 6, 6]",0,N
15,4,57,arrive,4,"[5, 0, 0, 0, 0, 0, 0, 4, 6, 4, 1, 0, 0, 6, 0, ...",2021-08-24 12:49:04,2021-08-24 13:00:17,"[13362, 2288, 3391, 3672, 4813, 3144, 5584, 64...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1561528,A170R0WAZC31WC,f,51,15.0,Variable data tech for print on demand,4,4,"[0, 6, 0, 6, 6]",0,N
20,5,22,cringe,5,"[3, 3, 0, 2, 0, 0, 1, 3, 6, 1, 1, 0, 2, 4, 1, ...",2021-08-24 13:23:42,2021-08-24 13:31:44,"[7343, 3488, 3920, 7568, 2992, 3808, 4968, 636...","[2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, ...",1183680,AAXX5LDVJ32F8,m,49,12.0,unemployed,5,5,"[0, 6, 0, 6, 6]",0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,1514,254,activate,872,"[5, 3, 1, 2, 4, 4, 3, 1, 1, 2, 2, 1, 0, 2, 6, ...",2021-08-28 09:38:19,2021-08-28 09:48:43,"[3744, 2720, 2944, 5064, 5416, 10400, 4312, 44...","[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, ...",7910923,ALTSDQ7JAWE7E,f,39,16.0,AR Specialist for my full time job and office ...,1514,872,"[0, 6, 0, 6, 6]",0,N
865,1515,45,start,873,"[6, 3, 6, 3, 2, 6, 2, 3, 3, 6, 3, 3, 6, 6, 3, ...",2021-08-28 09:44:10,2021-08-28 10:01:13,"[5328, 3789, 2640, 3636, 15144, 3160, 2187, 39...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",6194369,A1N35VV9QURKHX,f,45,14.0,Information Data Entry,1515,873,"[7, 6, 0, 6, 6]",0,N
1238,1516,35,wail,645,"[1, 0, 1, 0, 0, 0, 0, 0, 5, 0, 1, 0, 5, 3, 0, ...",2021-08-28 10:18:07,2021-08-28 10:32:48,"[6088, 3116, 5168, 33664, 3972, 4848, 3268, 34...","[1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, ...",6822510,A3O0ZQMDKOKAMR,f,44,12.0,I am a fulltime Decorator for a retail grocery...,1516,645,"[0, 6, 0, 6, 6]",0,N
672,1517,29,bathe,577,"[5, 0, 0, 0, 0, 0, 0, 1, 6, 2, 3, 0, 2, 5, 6, ...",2021-08-28 10:37:50,2021-08-28 10:53:09,"[4344, 2234, 2390, 9221, 20266, 128198, 21287,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, ...",3616355,A49A0FF6ZXYDU,f,53,16.0,library staff,1517,577,"[0, 6, 0, 6, 6]",0,N


In [20]:
## export to csv
#res_combined.to_csv(path+'results_clean_0813.csv', index=False)
res_combined.to_csv(path+'results_catchreject_0831.csv', index=False)

In [21]:
#######to export bad subjects list
df_bad = res_combined.loc[res_combined['badsubject']=='Y', ]
df_bad.to_csv(path+'badsubjects_0831.csv')